In [7]:
%run "..\general_functions\generalFunctions.ipynb"

In [ ]:
def genrateIndexListCat(scopeCategory, chartIndex=0, chartCount=5):

    """
    Generates a list of chart indices based on the scope category and chart parameters.

    Parameters:
    scopeCategory (list): A list representing the scope categories (e.g., sectors, segments, categories).
    chartIndex (int, optional): The starting index for the first chart. Default is 0.
    chartCount (int, optional): The number of charts to display on a full slide. Default is 5.

    Returns:
    list: A list containing the calculated indices for chart positioning.
    """
    if len(scopeCategory) == 0:
        return []
    
    # Calculate the number of pages needed
    page = len(scopeCategory)

    # Initialize the list to store chart indices
    lis = []
    
    # Loop to calculate indices until pages are exhausted
    while page > chartCount:
        lis.append(chartIndex)
        page -= chartCount
    # Append the remaining charts to the list
    lis.append(chartIndex)
    return [lis]

## PPTX Functions

#### Slide 1

In [ ]:
def process_bracket_data_slide1(dic,col="Base Price Bracket",bymanuf=False):
    ToplineBracket_brand = {}
    for key in dic.keys():
        df = dic[key].copy()
        df=DetectHeader(df)
        df=df[:-1]
        if col== "Base Price Bracket":
            df= df.rename(columns={"Base Price\xa0Bracket":"Base Price Bracket"})
        df[col] = df[col].ffill()
        if bymanuf:
            df[["Value Share"	,"Company WoB %"	,"Value Sales IYA"	,"Relative Price"]] = df[["Value Share"	,"Company WoB %"	,"Value Sales IYA"	,"Relative Price"]].replace(np.nan, 0).astype(float)
            if ManufOrTopC == "Manufacturer": df= df.rename(columns={"Manufacturer":"Top Companies"})
        else: 
            df[["Value Share"	,"Brand WoB %"	,"Value Sales IYA"	,"Relative Price"]] = df[["Value Share"	,"Brand WoB %"	,"Value Sales IYA"	,"Relative Price"]].replace(np.nan, 0).astype(float)
            if BrandOrTopB == "Brand": df= df.rename(columns={"Brand":"Top Brands"})
        df = df.sort_values('Value Share', ascending=False)
        df=df[~(df[col].str.contains('0-0'))]
        if df.shape[0]==0:
            print(key)
        else: 
            ToplineBracket_brand[key] = df

    return ToplineBracket_brand      

In [ ]:
def brandShareTopline(prs, modifiedShareToplineBracket, bracketsValue, numOfDuplicates, slide_by, clientElement, position=0):

    for slidenum in range(numOfDuplicates):
        market = list(modifiedShareToplineBracket.keys())[slidenum]
        df = modifiedShareToplineBracket[market].copy()
        df['Size'] =df[f"{slide_by}"].apply(lambda x: x.split('-')[1].split(' ')[0] if '-' in x else 9999).astype(str).str.extract('(\d+)').astype(float)

        def extract_size(value):
            if '6+' in value:
                return 6.5  # Assign higher value to ensure correct sorting
            elif '-' in value:
                return float(value.split('-')[0])  # Extract lower bound
            else:
                return 9999  # Fallback for unexpected cases

        df['Size'] = df[f"{slide_by}"].apply(extract_size)
        df = df.sort_values(by=['Size'], ascending=False)
        df=df.rename(columns={"Top Companies":"Top Brands", "Company WoB %":"Brand WoB %", "Brand":"Top Brands"})
        # Filter total brand data and clean up column values
        dfTotalBrand = df[df[f"{slide_by}"].str.contains('Total')]
        dfTotalBrand[f"{slide_by}"] = dfTotalBrand[f"{slide_by}"].str.replace(' Total', '')
        dfTotalBrand = dfTotalBrand[dfTotalBrand['Value Share'] > .0005]
        # Filter the main data frame based on size
        df = df[df['Size'].isin(dfTotalBrand['Size'].unique())].sort_values(by='Value Share', ascending=False)
        
        toplis = df[(df['Top Brands'].notna()) & (df['Top Brands'] != clientElement)]\
                    .drop_duplicates(subset='Top Brands')['Top Brands'].iloc[:3].to_list()

        otherList = ["Other", "Other Manufacturer", "Others", "All Others"]

        # Check if any element in toplis is in otherList
        if any(brand in otherList for brand in toplis):
            dfTopSales = df[(df['Top Brands'].notna()) & (df['Top Brands'] != clientElement)]\
                            .drop_duplicates(subset='Top Brands')['Top Brands'].iloc[:4].to_list()
        else:
            dfTopSales = df[(df['Top Brands'].notna()) & (df['Top Brands'] != clientElement)]\
                            .drop_duplicates(subset='Top Brands')['Top Brands'].iloc[:3].to_list()

        dfBrandInScope = df[df['Top Brands'].isin(dfTopSales)]
        
        # Calculate the 'Other' category for the data frame
        dfOther = df[(~df['Top Brands'].isin(dfTopSales + [clientElement])) & (~df[f"{slide_by}"].str.contains('Total'))].groupby([f"{slide_by}", 'Size'])['Value Share'].sum().reset_index().sort_values(by='Size', ascending=False)
        missingBracket = list(set(bracketsValue) - set(dfTotalBrand[f"{slide_by}"].unique()))
        missingOtherBracket = list(set(bracketsValue) - set(dfOther[f"{slide_by}"].unique()))
        missingBracket = pd.DataFrame({f"{slide_by}": missingBracket, 'Size': [float(re.search(f'\d+',x.split('-')[1].split(' ')[0]).group()) if '-' in x else 9999 for x in missingBracket]})
        missingOtherBracket = pd.DataFrame({f"{slide_by}": missingOtherBracket, 'Size': [float(re.search(f'\d+',x.split('-')[1].split(' ')[0]).group()) if '-' in x else 9999 for x in missingOtherBracket]})
        dfOther = pd.concat([dfOther, missingOtherBracket]).sort_values(by='Size', ascending=False)
        dfTotalBrand = pd.concat([dfTotalBrand, missingBracket]).sort_values(by='Size', ascending=False)

        
        # Filter the client's brand data
        dfClientBrand = df[df['Top Brands'] == clientElement]
        
        # Access slide shapes to update text and formatting
        shapes = prs.slides[slidenum + position].shapes

        
        headerNumber = get_shape_number(shapes, "Brand Share Topline By Size Bracket (Replace With SO WHAT)")
        titleNumber = get_shape_number(shapes, "Brand Share Topline By Size Bracket | National | Category | P12M")
        
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text = f'Brand Share Topline By {slide_by} | {market} | P12M'
        
        # Format text as bold and set font size
        shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
        for p in range(len(shapes[titleNumber].text_frame.paragraphs)):
            shapes[titleNumber].text_frame.paragraphs[p].font.size = Pt(12)
        
        shapes[headerNumber].text_frame.paragraphs[0].runs[0].text = shapes[headerNumber].text_frame.paragraphs[0].runs[0].text.replace('Size Bracket', slide_by)
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        # Create tables and charts
        tables, charts = createTableAndChart(shapes)
        # Adjust table row numbers
        table = tables[0].table
        num_rows_to_remove = len(table.rows) - dfTotalBrand[f"{slide_by}"].nunique() - 1
        for _ in range(num_rows_to_remove):
            if len(table.rows) > 1:  # Skip removing the first row if there is more than one row
                row = table.rows[1]
                remove_row(table, row)
        # Set table row height
        table_height = Inches(3.93)  # Specify the desired table height
        total_row_height = table_height - table.rows[0].height
        num_rows = len(table.rows) - 1  # Exclude the first row
        if num_rows > 0:
            cell_height = total_row_height / num_rows
            for row in range(1, len(table.rows)):
                table.rows[row].height = int(cell_height)
        
        # Replace the table data
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if i == 0:
                    # Update header cells
                    if j in [2, 3, 4]:
                        cell.text = cell.text.replace('Brand', clientElement)
                        for paragraph in cell.text_frame.paragraphs:
                            paragraph.font.name = 'Nexa Bold'
                            paragraph.font.size = Pt(8)
                            paragraph.alignment = PP_ALIGN.CENTER
                            paragraph.font.color.rgb = RGBColor(87, 85, 85)
                            paragraph.font.bold = False
                    continue
                
                # Update data cells
                sizeBracket = dfTotalBrand[f"{slide_by}"].unique()[i - 1]
                if j == 0:
                    cell.text = sizeBracket
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                
                if j == 3 or j == 4:
                    if j == 3:
                        value = dfClientBrand[dfClientBrand[f"{slide_by}"] == sizeBracket]['Value Sales IYA'].unique()
                        if len(value) == 0 or (dfClientBrand[dfClientBrand[f"{slide_by}"] == sizeBracket]['Brand WoB %'].empty or dfClientBrand[dfClientBrand[f"{slide_by}"] == sizeBracket]['Brand WoB %'].unique()[0] < .0005):
                             value = [0]

                        cell.text = '' if (len(value) == 0) or (int(round(float(value[0]) * 100, 0)) == 0) else (str(int(round(float(value[0]) * 100, 0))) + '%' if int(round(float(value[0]) * 100, 0)) <= 1000 else 'Large')
                    else:
                        value = dfClientBrand[dfClientBrand[f"{slide_by}"] == sizeBracket]['Relative Price'].unique()
                        if value.size > 0 and dfClientBrand[dfClientBrand[f"{slide_by}"] == sizeBracket]['Brand WoB %'].unique()[0] < .0005:
                            value = [0]
                        cell.text = '' if len(value) == 0 or (int(round(float(value[0]) * 100, 0)) == 0) else str(int(round(float(value[0]) * 100, 0))) + '%'
                    
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        
        # Update chart data
        for chartNum in [0, 1]:
            chart = charts[chartNum].chart
            chart_data = CategoryChartData()
            chart_data.categories = ['']
            missingBrandBracket = list(set(dfTotalBrand[f"{slide_by}"].unique()) - set(dfClientBrand[f"{slide_by}"].unique()))
            missingBrandBracket = pd.DataFrame({'Top Brands': clientElement, f"{slide_by}": missingBrandBracket, 'Size': [float(re.search(f'\d+',x.split('-')[1].split(' ')[0]).group()) if '-' in x else 9999 for x in missingBrandBracket]})
            dfClientBrand2 = pd.concat([dfClientBrand, missingBrandBracket]).sort_values(by='Size', ascending=False).replace(np.nan, None)
            dfClientBrand2 = dfClientBrand2.set_index(f"{slide_by}").reindex(dfTotalBrand[f"{slide_by}"]).reset_index()

            if chartNum == 0:
                # Exclude Value Share less than 0.05%
                dfClientBrand2['Brand WoB %'] = np.where(dfClientBrand2['Brand WoB %'] < .0005, None, dfClientBrand2['Brand WoB %'])
                brandWob = dfClientBrand2['Brand WoB %'].to_list()
                chart_data.add_series('Brand WoB %', brandWob)
            else:
                valueShare = dfTotalBrand['Value Share'].replace(np.nan, None).to_list()
                chart_data.add_series('Value Share', valueShare)
            chart.replace_data(chart_data)
                 # Update chart formatting
            xlsx_file=BytesIO()
            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 0,f"{slide_by}")

                worksheet.write_column(1, 0, dfClientBrand2[f"{slide_by}"], None)

            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())
        # Update the comparison chart
        chart2 = charts[2].chart
        chart_data2 = CategoryChartData()
        chart_data2.categories = dfTotalBrand[f"{slide_by}"].unique()
        missingBrandBracket = list(set(dfTotalBrand[f"{slide_by}"].unique()) - set(dfClientBrand[dfClientBrand['Top Brands'] == clientElement][f"{slide_by}"].unique()))
        missingBrandBracket = pd.DataFrame({'Top Brands': clientElement, f"{slide_by}": missingBrandBracket, 'Size': [float(re.search(f'\d+',x.split('-')[1].split(' ')[0]).group()) if '-' in x else 9999 for x in missingBrandBracket]})
        dfClientBrand2 = pd.concat([dfClientBrand[dfClientBrand['Top Brands'] == clientElement], missingBrandBracket]).sort_values(by='Size', ascending=False)
        dfClientBrand2 = dfClientBrand2.set_index(f"{slide_by}").reindex(dfTotalBrand[f"{slide_by}"]).reset_index()
        valueShare = dfClientBrand2['Value Share'].replace(np.nan, None).to_list()
        chart_data2.add_series(clientElement, valueShare)

        for brand in dfBrandInScope['Top Brands'].unique():
            missingBrandBracket = list(set(dfTotalBrand[f"{slide_by}"].unique()) - 
                                    set(dfBrandInScope[dfBrandInScope['Top Brands'] == brand][f"{slide_by}"].unique()))
            
            missingBrandBracket = pd.DataFrame({
                'Top Brands': brand,
                f"{slide_by}": missingBrandBracket,
                'Size': [float(re.search(r'\d+', x.split('-')[1].split(' ')[0]).group()) if '-' in x else 9999 
                        for x in missingBrandBracket]
            })
            dfClientBrand2 = pd.concat([dfBrandInScope[dfBrandInScope['Top Brands'] == brand], missingBrandBracket])\
                            .sort_values(by='Size', ascending=False)
            dfClientBrand2 = dfClientBrand2.set_index(f"{slide_by}")\
                                        .reindex(dfTotalBrand[f"{slide_by}"])\
                                        .reset_index()
            dfOther['Top Brands'] = "Others"
            otherList = ["Other", "Other Manufacturer", "Others", "All Others"]
            all_others_mask = dfClientBrand2['Top Brands'].isin(otherList)

            if all_others_mask.any():
                sum_all_others = dfClientBrand2.loc[all_others_mask, 'Value Share']
                dfClientBrand2 = dfClientBrand2[~all_others_mask]
                dfOther.loc[dfOther['Top Brands'] == "Others", 'Value Share'] = (
                    dfOther.loc[dfOther['Top Brands'] == "Others", 'Value Share'].values 
                    + sum_all_others.values  
                )
            if brand not in otherList :
                valueShare = dfClientBrand2['Value Share'].replace(np.nan, None).to_list()
                chart_data2.add_series(brand, valueShare)
        valueShare = dfOther['Value Share'].replace(np.nan, None).to_list()
        chart_data2.add_series('Others', valueShare)

        chart2.replace_data(chart_data2)



#### Slide 2 & 3

In [ ]:
def process_bracket_data(dic,col="Base Price Bracket",bymanuf=False):
    ToplineBracket_brand = {}
    for key in dic.keys():
        df = dic[key].copy()
        df=DetectHeader(df)
        df=df[:-1]
        if col== "Base Price Bracket":
            df= df.rename(columns={"Base Price\xa0Bracket":"Base Price Bracket"})
        df[col] = df[col].ffill()
        if bymanuf:
            df[["Value Share"	,"Company WoB %"	,"Value Sales IYA"	,"Relative Price"]] = df[["Value Share"	,"Company WoB %"	,"Value Sales IYA"	,"Relative Price"]].replace(np.nan, 0).astype(float)
            if ManufOrTopC == "Manufacturer": df= df.rename(columns={"Manufacturer":"Top Companies"})
        else: 
            df[["Value Share"	,"Brand WoB %"	,"Value Sales IYA"	,"Relative Price"]] = df[["Value Share"	,"Brand WoB %"	,"Value Sales IYA"	,"Relative Price"]].replace(np.nan, 0).astype(float)
            if BrandOrTopB == "Brand": df= df.rename(columns={"Brand":"Top Brands"})
        df = df.sort_values('Value Share', ascending=False)
        df=df[~(df[col].str.contains('0-0'))]
        ToplineBracket_brand[key] = df

    return ToplineBracket_brand        

In [ ]:
def selectDf(dicOfDf,marketScope,catScope,secpos):
    newDic = {key : dicOfDf[key] for key in dicOfDf.keys() if key.split(' | ')[0] in ([['National' ]+ marketScope][0] if national else marketScope)}
    out = splitDfs(newDic,catScope,index3forsplit[secpos])
    return out

In [ ]:
def repeatHierarchy(catScope,dictOfDf,dictOfDf_manuf,areaList,sectionPosition,slidetype,columnNumber):
    global position

    if catScope and areaList:
        catDfsplit2 = selectDf(dictOfDf,areaList,catScope,sectionPosition)
        catDfsplit2_manuf = selectDf(dictOfDf_manuf,areaList,catScope,sectionPosition)
        
        position = sum([duplication3[i]*(1 if type(index3[i])==int else len(index3[i])) for i in range(sectionPosition)])
        if ManufOn==True:
            for i,clientElement in enumerate(client_manuf):
                for dfs in catDfsplit2_manuf:
                    brandShareToplineBySector(prs3,dictOfDf_manuf,dfs,clientElement,1,retailerLis=retailer,channelLis=channel,customLis=custom,slideType=slidetype, slide_by = columnName[columnNumber],position=position,manufOrbrand="Top Companies")
                    position=int(position+1)        
        for i,clientElement in enumerate(client_brands):
            for dfs in catDfsplit2:
                brandShareToplineBySector(prs3,dictOfDf,dfs,clientElement,1,retailerLis=retailer,channelLis=channel,customLis=custom,slideType=slidetype, slide_by = columnName[columnNumber],position=position,manufOrbrand="Top Brands")
                position=int(position+1)
        sectionPosition += 1
    else:
        print('We do not have this category level or Market Level')
    print(position)

    return position,sectionPosition

In [ ]:
def repeatWithoutRetail(catScope,dictOfDf,dictOfDf_manuf,sectionPosition,slidetype,columnNumber):
    global position

    if catScope:
        catDfsplit2 = splitDfs(dictOfDf,dfLis,index2forsplit[sectionPosition])

        position = sum([duplication2[i]*(1 if type(index2[i])==int else len(index2[i])) for i in range(sectionPosition)])
        retailerLis=''
        channelLis=''
        customLis=''
        if ManufOn==True:
            for i,clientElement in enumerate(client_manuf):
                for dfs in catDfsplit2:
                    brandShareToplineBySector(prs2,dictOfDf_manuf,dfs,clientElement,1,retailerLis,channelLis,customLis,slideType=slidetype, slide_by = columnName[columnNumber],position=position,manufOrbrand="Top Companies")
                    position=int(position+1)
        for i,clientElement in enumerate(client_brands):
            for dfs in catDfsplit2:
                brandShareToplineBySector(prs2,dictOfDf,dfs,clientElement,1,retailerLis,channelLis,customLis,slideType=slidetype, slide_by = columnName[columnNumber],position=position,manufOrbrand="Top Brands")
                position=int(position+1)
        sectionPosition += 1
    else:
        print('We do not have this category level')
    print(position)
    
    

    return position,sectionPosition

In [ ]:
import re

def brandShareToplineBySector(prs, ToplineBracketSector, markets, clientBrand, numOfDuplicates, retailerLis='', channelLis='', customLis='', slideType='', slide_by='', position=0, manufOrbrand="Top Brands"):
    '''
    Set tables and chart order according to each slide type.
    The order of the table is not the same as in the slide base same for the chart.

    Parameters:
        prs (pptx.presentation.Presentation): The PowerPoint presentation object where slides will be added or modified.
        ToplineBracketSector (dict): Dictionary containing data frames for different market sectors.
        markets (list): List of market names.
        clientBrand (str): Name of the client brand to highlight in the presentation.
        numOfDuplicates (int): Number of slides to create or modify.
        retailerLis (str, optional): List of retailers. Default is an empty string.
        channelLis (str, optional): List of channels. Default is an empty string.
        customLis (str, optional): List of customs. Default is an empty string.
        slideType (str, optional): Type of slide. Default is an empty string.
        slide_by (str, optional): Column name used to determine the size bracket. Default is an empty string.
        position (int, optional): Position offset for the slides. Default is 0.
    '''

    # Function to extract the numeric value for sorting
    def extract_numeric_value(value):
        # Extract the first numeric part of the string (e.g., "3-3.5 €" -> 3.5)
        match = re.search(r'\d+\.?\d*', value)
        if match:
            return float(match.group())
        return 9999  # Default value for sorting if no numeric part is found

    for slidenum in range(numOfDuplicates):
        marketSplit = markets[slidenum].split(' | ')
        market = marketSplit[0]
        cat = marketSplit[1]

        # Get all the shapes in the slide, Tables, Title, charts, etc.
        slide = prs.slides[slidenum + position]
        shapes = prs.slides[slidenum + position].shapes
        tables, charts = createTableAndChart(shapes)
        tablelength = len(markets)  # tablesOrders[len(tables)]

        title_num = get_shape_number(shapes, "Brand Share Topline By Size Bracket | By Sector | National | P12M")
        headerNumber = get_shape_number(shapes, "Size Bracket by Sector/Segment (Replace With SO WHAT)")
        data_source_num = title_num - 1

        shapes[data_source_num].text = data_source

        if retailerLis != '' or channelLis != '' or customLis != '':
            area = 'By Retailer' if (market in retailerLis) else 'By National' if ('National' == market) else 'By Channel' if (market in channelLis) else 'By ' + customareas
            shapes[title_num].text = f"Brand Share Topline By {slide_by} | {area} | {cat} | P12M"
            shapes[headerNumber].text_frame.paragraphs[0].runs[0].text = shapes[headerNumber].text_frame.paragraphs[0].runs[0].text.replace('Sector/Segment', 'Channel/Retailer')

            for p in range(len(shapes[title_num].text_frame.paragraphs)):
                shapes[title_num].text_frame.paragraphs[p].font.bold = True
                shapes[title_num].text_frame.paragraphs[p].font.size = Pt(12)
        else:
            shapes[title_num].text = f"Brand Share Topline By {slide_by} | By {slideType} | {market} | P12M"

            for p in range(len(shapes[title_num].text_frame.paragraphs)):
                shapes[title_num].text_frame.paragraphs[p].font.bold = True
                shapes[title_num].text_frame.paragraphs[p].font.size = Pt(12)

        shapes[headerNumber].text_frame.paragraphs[0].runs[0].text = shapes[headerNumber].text_frame.paragraphs[0].runs[0].text.replace('Size Bracket', slide_by)
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)

        # Concat all sector or segment for the same retailer
        dfSectors = pd.DataFrame()
        sectors = [sector for sector in ToplineBracketSector.keys() if (market == sector.split(' | ')[0] or market == sector.split(' | ')[1])]

        for sector in markets:
            dfSector = ToplineBracketSector[sector][ToplineBracketSector[sector][slide_by].str.contains('Total')]
            dfSector['KEY'] = sector
            dfSectors = pd.concat([dfSectors, dfSector])

        dfSectors[slide_by] = dfSectors[slide_by].str.replace(' Total', '')
        dfSectors['Size'] = dfSectors[slide_by].apply(extract_numeric_value)
        dfSectors = dfSectors.sort_values(by='Size', ascending=False)

        table0 = tables[0].table

        num_rows_to_remove = len(table0.rows) - dfSectors[slide_by].nunique() - 2
        num_cols_to_remove = (4 - tablelength) * 2

        table0 = removeRowFromTable(table0, num_rows_to_remove, rowToExclude=2)
        table0 = col_cell_remove_no_height_adjusted(table0, num_cols_to_remove)

        for i, row in enumerate(table0.rows):
            for j, cell in enumerate(row.cells):
                if i > 1 and j == 0:  # Exclude first two rows
                    cell.text = dfSectors[slide_by].unique()[i - 2]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                if i == 1 and j > 0 and cell.text != "Value\nShare (%)":
                    text_frame = cell.text_frame
                    cell.text = cell.text.replace('Findus', clientBrand).replace('| WOB', '').replace('Value Share', 'VS')
                    paragraph = text_frame.paragraphs[0]
                    run_2 = paragraph.add_run()
                    run_2.text = "| WOB"
                    run_2.font.color.rgb = RGBColor(0, 160, 151)
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        for chartNum in range(8):
            if chartNum >= (2 * tablelength):
                delete_table_chart(slide, charts[chartNum])

        for sec in range(len(markets)):
            dfSector = ToplineBracketSector[markets[sec]]
            dfSector['Size'] = dfSector[slide_by].apply(extract_numeric_value)
            # dfSector = dfSector[dfSector['Size'] != 0]

            dfSectorTotal = dfSector[dfSector[slide_by].str.contains('Total')]
            dfSectorTotal[slide_by] = dfSectorTotal[slide_by].str.replace(' Total', '')

            dfSectorBrand = dfSector[(dfSector[manufOrbrand] == clientBrand) & (dfSector[slide_by].isin(dfSectorTotal[slide_by].unique()))]

            missingTotalBracket = list(set(dfSectors[slide_by].unique()) - set(dfSectorTotal[slide_by].unique()))
            missingTotalBracket = pd.DataFrame({slide_by: missingTotalBracket, 'Size': [extract_numeric_value(x) for x in missingTotalBracket]})
            dfSectorTotal = pd.concat([dfSectorTotal, missingTotalBracket]).sort_values(by='Size', ascending=False)

            missingBrandBracket = list(set(dfSectors[slide_by].unique()) - set(dfSectorBrand[slide_by].unique()))
            missingBrandBracket = pd.DataFrame({slide_by: missingBrandBracket, 'Size': [extract_numeric_value(x) for x in missingBrandBracket]})
            dfSectorBrand = pd.concat([dfSectorBrand, missingBrandBracket]).sort_values(by='Size', ascending=False)

            chartNum = 2 * sec
            chart = charts[chartNum].chart
            chart_data = CategoryChartData()
            chart_data.categories = ['']
            dfSectorTotal['Value Share'] = dfSectorTotal['Value Share'].astype(float)
            dfSectorTotal['Value Share'] = np.where(round(dfSectorTotal['Value Share'].replace(np.nan, 0), 3) < .0005, None, dfSectorTotal['Value Share'])
            series = dfSectorTotal['Value Share'].replace(np.nan, None)
            chart_data.add_series(markets[sec], series)
            chart.replace_data(chart_data)
            xlsx_file=BytesIO()
            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 0,f"{slide_by}")

                worksheet.write_column(1, 0, dfSectorTotal[slide_by], None)
            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

            chartNum = 2 * sec + 1
            chart = charts[chartNum].chart
            chart_data = CategoryChartData()
            chart_data.categories = ['']
            dfSectorBrand['Value Share'] = dfSectorBrand['Value Share'].astype(float)
            dfSectorBrand['Value Share'] = np.where(round(dfSectorBrand['Value Share'].replace(np.nan, 0), 3) < .0005, None, dfSectorBrand['Value Share'])
            series = dfSectorBrand['Value Share'].replace(np.nan, None)
            chart_data.add_series(markets[sec], series)
            chart.replace_data(chart_data)
            xlsx_file=BytesIO()
            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 0,f"{slide_by}")

                worksheet.write_column(1, 0, dfSectorBrand[slide_by], None)

            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())
            for i, row in enumerate(table0.rows):
                for j, cell in enumerate(row.cells):
                    if i==0 and j==(sec)*2+1:
                        # if slideType =='By Channel':
                        if 'By' in slideType:
                            cell.text=markets[sec].split(' | ')[0] #if (sectors[sec].split(' | ')[0] in retailerLis) or (sectors[sec].split(' | ')[0] in channelLis) else sectors[sec].split(' | ')[1]
                        else:
                            cell.text=markets[sec].split(' | ')[1]#[:10]
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(10) #Changed 1
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
                        cell.text_frame.paragraphs[0].font.bold = False
                        
                    if i>1 and j==(sec)*2+2:
                        if manufOrbrand=="Top Brands":
                            value='' if (dfSectorBrand['Brand WoB %'].reset_index(drop=True).astype(str)[i-2]=='nan'  or dfSectorBrand['Value Share'].reset_index(drop=True).astype(float)[i-2]<0.0005)  else str(int(round(dfSectorBrand['Brand WoB %'].reset_index(drop=True).astype(float)[i-2]*100,0)))+'%'
                        else:
                            value='' if (dfSectorBrand['Company WoB %'].reset_index(drop=True).astype(str)[i-2]=='nan' or dfSectorBrand['Value Share'].reset_index(drop=True).astype(float)[i-2]<0.0005)  else str(int(round(dfSectorBrand['Company WoB %'].reset_index(drop=True).astype(float)[i-2]*100,0)))+'%'

                        cell.text='' if (value =='0.0%' or value =='0%') else value
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                        cell.text_frame.paragraphs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.RIGHT
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0,160,151)
                        cell.text_frame.paragraphs[0].font.bold = True


#### Slide 4

In [ ]:
def process_bracket_data_cat(dic,col="", col2=""):
    ToplineBracketSecSeg_brand_cat={}
    for k in dic.keys():
        t = dic[k].copy()
        t=DetectHeader(t)
        t=t[:-1]
        if col== "Base Price Bracket":
            t= t.rename(columns={"Base Price\xa0Bracket":"Base Price Bracket"})
        if ManufOrTopC == "Manufacturer": t= t.rename(columns={"Manufacturer":"Top Brands"})
        if ManufOrTopC == "Top Companies": t= t.rename(columns={"Top Companies":"Top Brands"})
        if BrandOrTopB == "Brand": t= t.rename(columns={"Brand":"Top Brands"})
        t[col] = t[col].ffill()
        t[col2] = t[col2].ffill()
        t=t[~(t[col].str.contains('0-0'))]
        
        ToplineBracketSecSeg_brand_cat[k] = t
    return ToplineBracketSecSeg_brand_cat     

In [ ]:
def repeatCatScope(catScope,dictOfDf,dictOfDf_manuf,sectionPosition,slidetype,columnNumber):
    global position

    if catScope:
        catDfsplit3 = splitDfs(dictOfDf,dfLis,[1])
        numOfDuplicates=math.ceil(len(catScope)/5)

        position = sum([duplication4[i]*(1 if type(index4[i])==int else len(index4[i])) for i in range(sectionPosition)])
        if ManufOn:
            for i,clientElement in enumerate(client_manuf):
                for dfs in catDfsplit3:
                    brandShareCatScope(prs4,dictOfDf_manuf,dfs,clientElement,numOfDuplicates, catScope, categories[0],slideType=slidetype, slide_by = columnName[columnNumber],position=position,manufOrbrand="Top Companies")#only work with 1 cat
                    position=int(position+numOfDuplicates)
        for i,clientElement in enumerate(client_brands):
            for dfs in catDfsplit3:
                brandShareCatScope(prs4,dictOfDf,dfs,clientElement,numOfDuplicates, catScope, categories[0],slideType=slidetype, slide_by = columnName[columnNumber],position=position,manufOrbrand="Top Brands")#only work with 1 cat
                position=int(position+numOfDuplicates)
        sectionPosition += 1
    else:
        print('We do not have this category level')
        print(position)
        
    

    return position,sectionPosition

In [ ]:
import re

def brandShareCatScope(prs, ToplineBracketSector, markets, clientBrand, numOfDuplicates, scope, category='', slideType='', slide_by='', position=0, manufOrbrand="Top Brands"):
    '''
    Set tables and chart order according to each slide type.
    The order of the table is not the same as in the slide base same for the chart.

    Parameters:
        prs (pptx.presentation.Presentation): The PowerPoint presentation object where slides will be added or modified.
        ToplineBracketSector (dict): Dictionary containing data frames for different market sectors.
        markets (list): List of market names.
        clientBrand (str): Name of the client brand to highlight in the presentation.
        numOfDuplicates (int): Number of slides to create or modify.
        scope (list): List of scope values.
        category (str, optional): Category name. Default is an empty string.
        slideType (str, optional): Type of slide. Default is an empty string.
        slide_by (str, optional): Column name used to determine the size bracket. Default is an empty string.
        position (int, optional): Position offset for the slides. Default is 0.
        manufOrbrand (str, optional): Manufacturer or brand type. Default is "Top Brands".
    '''

    # Function to extract the numeric value for sorting
    def extract_numeric_value(value):
        # Extract the first numeric part of the string (e.g., "3-3.5 €" -> 3.5)
        match = re.search(r'\d+\.?\d*', value)
        if match:
            return float(match.group())
        return 9999  # Default value for sorting if no numeric part is found

    for slidenum in range(numOfDuplicates):
        market = markets[0]

        # Get all the shapes in the slide, Tables, Title, charts, etc.
        slide = prs.slides[slidenum + position]
        shapes = slide.shapes
        tables, charts = createTableAndChart(shapes)
        tablelength = min(5, len(scope) - 5 * slidenum)

        title_num = get_shape_number(shapes, "Brand Share Topline By Size Bracket By Sector | National | Neumarkt | P12M")
        headerNumber = get_shape_number(shapes, "Brackets Analysis By Sector (Replace with SO WHAT)")
        data_source_num = title_num - 1

        shapes[data_source_num].text = data_source

        shapes[title_num].text = f"Brand Share Topline By {slide_by} By {slideType} | {market} | {clientBrand} | P12M"
        shapes[title_num].text_frame.paragraphs[0].font.bold = True
        shapes[title_num].text_frame.paragraphs[0].font.size = Pt(12)

        shapes[headerNumber].text_frame.paragraphs[0].runs[0].text = shapes[headerNumber].text_frame.paragraphs[0].runs[0].text.replace('Sector', slideType)
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)

        # Concat all sector or segment for the same retailer
        dfTot = pd.DataFrame()
        dfSectorsTot = pd.DataFrame()
        dfTot = ToplineBracketSector[market][ToplineBracketSector[market][slide_by].str.contains('Total')]
        dfSectorsTot = ToplineBracketSector[market][(ToplineBracketSector[market][slideType].str.contains('Total')) & ~ToplineBracketSector[market][slide_by].str.contains('Total')]
        maxaxis = round(math.ceil(max(dfTot["Value Share"]) * 1.1 * 10) / 10, 1)
        maxaxis = max(maxaxis, round(math.ceil(max(dfSectorsTot["Value Share"]) * 1.1 * 10) / 10, 1))

        dfTot[slide_by] = dfTot[slide_by].str.replace(' Total', '')
        dfSectorsTot[slideType] = dfSectorsTot[slideType].str.replace(' Total', '')
        dfTot['Scope'] = "Total"
        dfSectorsTot['Scope'] = slideType
        dfTot['Size'] = dfTot[slide_by].apply(extract_numeric_value)
        dfSectorsTot['Size'] = dfSectorsTot[slide_by].apply(extract_numeric_value)
        dfTot = dfTot.sort_values(by='Size', ascending=False)
        dfSectorsTot = dfSectorsTot.sort_values(by='Size', ascending=False)

        table0 = tables[0].table
        num_rows_to_remove = len(table0.rows) - dfTot[slide_by].nunique() - 2
        num_cols_to_remove = (5 - tablelength) * 2

        table0 = removeRowFromTable(table0, num_rows_to_remove, rowToExclude=2)
        table0 = col_cell_remove_no_height_adjusted(table0, num_cols_to_remove)

        for i, row in enumerate(table0.rows):
            for j, cell in enumerate(row.cells):
                if i == 0 and j == 1:
                    cell.text = category
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                    cell.text_frame.paragraphs[0].font.bold = False
                if i == 0 and j == 2 and tablelength >= 1:
                    cell.text = scope[0 + 5 * slidenum]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                    cell.text_frame.paragraphs[0].font.bold = False
                if i == 0 and j == 4 and tablelength >= 2:
                    cell.text = scope[1 + 5 * slidenum]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                    cell.text_frame.paragraphs[0].font.bold = False
                if i == 0 and j == 6 and tablelength >= 3:
                    cell.text = scope[2 + 5 * slidenum]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                    cell.text_frame.paragraphs[0].font.bold = False
                if i == 0 and j == 8 and tablelength >= 4:
                    cell.text = scope[3 + 5 * slidenum]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                    cell.text_frame.paragraphs[0].font.bold = False
                if i == 0 and j == 10 and tablelength >= 5:
                    cell.text = scope[4 + 5 * slidenum]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                    cell.text_frame.paragraphs[0].font.bold = False
                if i > 1 and j == 0:  # Exclude first two rows
                    cell.text = dfTot[slide_by].unique()[i - 2]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                if i == 1 and j > 0:
                    text_frame = cell.text_frame

                    if cell.text == "Value Share (%)":
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                    if cell.text != "Value Share (%)":
                        cell.text = cell.text.replace('Neumarkt', clientBrand).replace('| WOB', '')
                        paragraph = text_frame.paragraphs[0]
                        run_2 = paragraph.add_run()
                        run_2.text = "| WOB"
                        run_2.font.color.rgb = RGBColor(0, 160, 151)
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        for chartNum in range(11):
            if chartNum > (2 * tablelength):
                delete_table_chart(slide, charts[chartNum])
            if chartNum == 0:
                chart = charts[chartNum].chart
                chart_data = CategoryChartData()
                chart_data.categories = ['']
                dfTot['Value Share'] = dfTot['Value Share'].astype(float)
                dfTot['Value Share'] = np.where(round(dfTot['Value Share'].replace(np.nan, 0), 3) < .0005, None, dfTot['Value Share'])
                series = dfTot['Value Share'].replace(np.nan, None)
                chart_data.add_series(category, series)
                chart.replace_data(chart_data)
                value_axis = chart.value_axis
                value_axis.minimum_scale = 0
                value_axis.maximum_scale = maxaxis
                xlsx_file = BytesIO()
                with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                    chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                    worksheet.write(0, 0, f"{slide_by}")
                    worksheet.write_column(1, 0, dfTot[slide_by], None)
                chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        for secnum, sec in enumerate(scope):
            if math.floor(secnum / 5) == slidenum:
                secnum = secnum - 5 * slidenum
                dfSectorBrand = pd.DataFrame()
                dfSectors = pd.DataFrame()
                dfSectorBrand = ToplineBracketSector[market][(ToplineBracketSector[market][slideType] == sec) & (ToplineBracketSector[market]["Top Brands"] == clientBrand)]
                dfSectorBrand['Size'] = dfSectorBrand[slide_by].apply(extract_numeric_value)
                # dfSectorBrand = dfSectorBrand[dfSectorBrand['Size'] != 0]
                dfSectorBrand['Scope'] = sec + " " + clientBrand
                dfSectors = dfSectorsTot[dfSectorsTot[slideType] == sec]

                # Missing brackets scope vs category
                missingTotalBracket = list(set(dfTot[slide_by].unique()) - set(dfSectors[slide_by].unique()))
                missingTotalBracket = pd.DataFrame({slide_by: missingTotalBracket, 'Size': [extract_numeric_value(x) for x in missingTotalBracket]})
                dfSectors = pd.concat([dfSectors, missingTotalBracket]).sort_values(by='Size', ascending=False)
                # Missing brackets brands vs category
                missingBrandBracket = list(set(dfTot[slide_by].unique()) - set(dfSectorBrand[slide_by].unique()))
                missingBrandBracket = pd.DataFrame({slide_by: missingBrandBracket, 'Size': [extract_numeric_value(x) for x in missingBrandBracket]})
                dfSectorBrand = pd.concat([dfSectorBrand, missingBrandBracket]).sort_values(by='Size', ascending=False)

                for i, row in enumerate(table0.rows):
                    for j, cell in enumerate(row.cells):
                        if i>1 and j==(secnum)*2+3 and tablelength>=secnum+1:
                            if manufOrbrand=="Top Brands":
                                value='' if (dfSectorBrand['Brand WoB %'].reset_index(drop=True).astype(str)[i-2]=='nan' or dfSectorBrand['Value Share'].reset_index(drop=True)[i-2]<0.0005) else str(int(round(dfSectorBrand['Brand WoB %'].reset_index(drop=True).astype(float)[i-2]*100,0)))+'%'
                            else:
                                value='' if (dfSectorBrand['Company WoB %'].reset_index(drop=True).astype(str)[i-2]=='nan' or dfSectorBrand['Value Share'].reset_index(drop=True)[i-2]<0.0005) else str(int(round(dfSectorBrand['Company WoB %'].reset_index(drop=True).astype(float)[i-2]*100,0)))+'%'

                            cell.text='' if (value =='0.0%' or value =='0%') else value
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Book Italic'
                            cell.text_frame.paragraphs[0].font.size = Pt(6)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.RIGHT
                            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0,160,151)
                            cell.text_frame.paragraphs[0].font.bold = True

                for chartNum in range(11):
                    if chartNum > 0 and chartNum - 2 * secnum == 1:
                        chart = charts[chartNum].chart
                        chart_data = CategoryChartData()
                        chart_data.categories = ['']
                        dfSectors['Value Share'] = dfSectors['Value Share'].astype(float)
                        dfSectors['Value Share'] = np.where(round(dfSectors['Value Share'].replace(np.nan, 0), 3) < .0005, None, dfSectors['Value Share'])
                        series = dfSectors['Value Share'].replace(np.nan, None)
                        chart_data.add_series(sec, series)
                        chart.replace_data(chart_data)
                        value_axis = chart.value_axis
                        value_axis.minimum_scale = 0
                        value_axis.maximum_scale = maxaxis

                        xlsx_file = BytesIO()
                        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                            worksheet.write(0, 0, f"{slide_by}")
                            worksheet.write_column(1, 0, dfSectors[slide_by], None)
                        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

                    if chartNum > 0 and chartNum - 2 * secnum == 2:
                        chart = charts[chartNum].chart
                        chart_data = CategoryChartData()
                        chart_data.categories = ['']
                        dfSectorBrand['Value Share'] = dfSectorBrand['Value Share'].astype(float)
                        dfSectorBrand['Value Share'] = np.where(round(dfSectorBrand['Value Share'].replace(np.nan, 0), 3) < .0005, None, dfSectorBrand['Value Share'])
                        series = dfSectorBrand['Value Share'].replace(np.nan, None)
                        chart_data.add_series(sec, series)
                        chart.replace_data(chart_data)
                        value_axis = chart.value_axis
                        value_axis.minimum_scale = 0
                        value_axis.maximum_scale = maxaxis
                        xlsx_file = BytesIO()
                        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                            worksheet.write(0, 0, f"{slide_by}")
                            worksheet.write_column(1, 0, dfSectorBrand[slide_by], None)
                        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

#### Slide 5

In [ ]:
def process_bracket_data_parent(dic,col="", col2="", col3=""):
    ToplineBracketSecSeg_brand_parent={}
    ToplineBracketSecSeg_parentnumber={}
    for k in dic.keys():
        t = dic[k].copy()
        t=DetectHeader(t)
        t=t[:-1]

        if col== "Base Price Bracket":
            t= t.rename(columns={"Base Price\xa0Bracket":"Base Price Bracket"})
        if ManufOrTopC == "Manufacturer": t= t.rename(columns={"Manufacturer":"Top Brands"})
        if ManufOrTopC == "Top Companies": t= t.rename(columns={"Top Companies":"Top Brands"})
        if BrandOrTopB == "Brand": t= t.rename(columns={"Brand":"Top Brands"})
        t[col] = t[col].ffill()
        t[col2] = t[col2].ffill()
        t[col3] = t[col3].ffill()
        t=t[~(t[col].str.contains('0-0'))]
        ttot=t[col2][t[col2].str.contains('Total')].drop_duplicates()

        ttot=ttot.str.replace(' Total','')
        ttot=ttot[ttot.isin(stringtodic[col2])].reset_index(drop=True)
        ToplineBracketSecSeg_brand_parent[k] = t
        ToplineBracketSecSeg_parentnumber[k]=ttot
    return ToplineBracketSecSeg_brand_parent, ToplineBracketSecSeg_parentnumber

In [ ]:
def repeatParentScope(catScope,numparent, parent,dictOfDf,dictOfDf_manuf,sectionPosition,slidetype,columnNumber):
    global position

    if catScope:
        catDfsplit3 = splitDfs(dictOfDf,dfLis,[1])
        numOfDuplicates=math.ceil(len(catScope)/5)
        position = sum([duplication5[i]*(1 if type(index5[i])==int else len(index5[i])) for i in range(sectionPosition)])
        if ManufOn:
            for i,clientElement in enumerate(client_manuf):
                for dfs in catDfsplit3:
                    
                    parentlist=numparent[dfs[0]]
                    brandShareParentScope(prs5,dictOfDf_manuf,dfs,parent,parentlist,clientElement,numOfDuplicates, catScope, categories[0],slideType=slidetype, slide_by = columnName[columnNumber],position=position,manufOrbrand="Top Companies")#only work with 1 cat
                    position=int(position+len(parentlist)*numOfDuplicates)
        for i,clientElement in enumerate(client_brands):
            for dfs in catDfsplit3:

                parentlist=numparent[dfs[0]]
                brandShareParentScope(prs5,dictOfDf,dfs,parent,parentlist,clientElement,numOfDuplicates, catScope, categories[0],slideType=slidetype, slide_by = columnName[columnNumber],position=position,manufOrbrand="Top Brands")#only work with 1 cat
                position=int(position+len(parentlist)*numOfDuplicates)
        sectionPosition += 1
    else:
        print('We do not have this category level')
        print(position)
        
    

    return position,sectionPosition

In [ ]:
import re

def brandShareParentScope(prs, ToplineBracketSector, markets, parent, parentlist, clientBrand, numOfDuplicates, scope, category='', slideType='', slide_by='', position=0, manufOrbrand="Top Brands"):
    '''
    Set tables and chart order according to each slide type.
    The order of the table is not the same as in the slide base same for the chart.

    Parameters:
        prs (pptx.presentation.Presentation): The PowerPoint presentation object where slides will be added or modified.
        ToplineBracketSector (dict): Dictionary containing data frames for different market sectors.
        markets (list): List of market names.
        parent (str): Parent category.
        parentlist (list): List of parent values.
        clientBrand (str): Name of the client brand to highlight in the presentation.
        numOfDuplicates (int): Number of slides to create or modify.
        scope (list): List of scope values.
        category (str, optional): Category name. Default is an empty string.
        slideType (str, optional): Type of slide. Default is an empty string.
        slide_by (str, optional): Column name used to determine the size bracket. Default is an empty string.
        position (int, optional): Position offset for the slides. Default is 0.
        manufOrbrand (str, optional): Manufacturer or brand type. Default is "Top Brands".
    '''

    # Function to extract the numeric value for sorting
    def extract_numeric_value(value):
        # Extract the first numeric part of the string (e.g., "3-3.5 €" -> 3.5)
        match = re.search(r'\d+\.?\d*', value)
        if match:
            return float(match.group())
        return 9999  # Default value for sorting if no numeric part is found

    market = markets[0]
    for numpar, namepar in enumerate(parentlist):
        for slidenum in range(numOfDuplicates):
            # Get all the shapes in the slide, Tables, Title, charts, etc.
            slide = prs.slides[position + (numpar) * numOfDuplicates + slidenum]
            shapes = slide.shapes
            tables, charts = createTableAndChart(shapes)
            tablelength = min(5, len(scope) - 5 * slidenum)

            title_num = get_shape_number(shapes, "Brand Share Topline By Size Bracket By Segment | National | Neumarkt | P12M")
            headerNumber = get_shape_number(shapes, "Brackets Analysis By Segment (Replace with SO WHAT)")
            data_source_num = title_num - 1

            shapes[data_source_num].text = data_source

            shapes[title_num].text = f"Brand Share Topline By {slide_by} By {slideType} | {market} | {clientBrand} | P12M"
            shapes[title_num].text_frame.paragraphs[0].font.bold = True
            shapes[title_num].text_frame.paragraphs[0].font.size = Pt(12)

            shapes[headerNumber].text_frame.paragraphs[0].runs[0].text = shapes[headerNumber].text_frame.paragraphs[0].runs[0].text.replace('Segment', parent + " X " + slideType)
            shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)

            # Concat all sector or segment for the same retailer
            dfTot = pd.DataFrame()
            dfTotParent = pd.DataFrame()
            dfSectorsTot = pd.DataFrame()
            dfTot = ToplineBracketSector[market][ToplineBracketSector[market][slide_by].str.contains('Total')]
            dfTotParent = ToplineBracketSector[market][(ToplineBracketSector[market][parent].str.contains('Total')) & ~ToplineBracketSector[market][slide_by].str.contains('Total')]
            dfSectorsTot = ToplineBracketSector[market][(ToplineBracketSector[market][slideType].str.contains('Total')) & ~ToplineBracketSector[market][slide_by].str.contains('Total') & ~ToplineBracketSector[market][parent].str.contains('Total')]
            maxaxis = round(math.ceil(max(dfTot["Value Share"]) * 1.1 * 10) / 10, 1)
            maxaxis = max(maxaxis, round(math.ceil(max(dfSectorsTot["Value Share"]) * 1.1 * 10) / 10, 1))
            maxaxis = max(maxaxis, round(math.ceil(max(dfTotParent["Value Share"]) * 1.1 * 10) / 10, 1))

            dfTot[slide_by] = dfTot[slide_by].str.replace(' Total', '')
            dfTotParent[parent] = dfTotParent[parent].str.replace(' Total', '')
            dfTotParent = dfTotParent[dfTotParent[parent] == namepar]
            dfSectorsTot[slideType] = dfSectorsTot[slideType].str.replace(' Total', '')
            dfSectorsTot = dfSectorsTot[dfSectorsTot[parent] == namepar]
            dfTot['Scope'] = "Total"
            dfTotParent['Scope'] = "Parent"
            dfSectorsTot['Scope'] = slideType
            dfTot['Size'] = dfTot[slide_by].apply(extract_numeric_value)
            dfTotParent['Size'] = dfTotParent[slide_by].apply(extract_numeric_value)
            dfSectorsTot['Size'] = dfSectorsTot[slide_by].apply(extract_numeric_value)
            dfTot = dfTot.sort_values(by='Size', ascending=False)
            dfTotParent = dfTotParent.sort_values(by='Size', ascending=False)
            dfSectorsTot = dfSectorsTot.sort_values(by='Size', ascending=False)
            VStotal = dfTot['Value Sales'].sum()
            dfTot['Value Share recal'] = (dfTot['Value Sales'] / VStotal)

            # Missing brackets parent vs category
            missingParentBracket = list(set(dfTot[slide_by].unique()) - set(dfTotParent[slide_by].unique()))
            missingParentBracket = pd.DataFrame({slide_by: missingParentBracket, 'Size': [extract_numeric_value(x) for x in missingParentBracket]})
            dfTotParent = pd.concat([dfTotParent, missingParentBracket]).sort_values(by='Size', ascending=False)

            table0 = tables[0].table
            num_rows_to_remove = len(table0.rows) - dfTot[slide_by].nunique() - 2
            num_cols_to_remove = (5 - tablelength) * 2

            table0 = removeRowFromTable(table0, num_rows_to_remove, rowToExclude=2)
            table0 = col_cell_remove_no_height_adjusted(table0, num_cols_to_remove)

            for i, row in enumerate(table0.rows):
                for j, cell in enumerate(row.cells):
                    if i == 0 and j == 1:
                        cell.text = category
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = False
                    if i == 0 and j == 2:
                        cell.text = namepar
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = False
                    if i == 0 and j == 3 and tablelength >= 1:
                        cell.text = scope[0 + 5 * slidenum]
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = False
                    if i == 0 and j == 5 and tablelength >= 2:
                        cell.text = scope[1 + 5 * slidenum]
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = False
                    if i == 0 and j == 7 and tablelength >= 3:
                        cell.text = scope[2 + 5 * slidenum]
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = False
                    if i == 0 and j == 9 and tablelength >= 4:
                        cell.text = scope[3 + 5 * slidenum]
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = False
                    if i == 0 and j == 11 and tablelength >= 5:
                        cell.text = scope[4 + 5 * slidenum]
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = False
                    if i > 1 and j == 0:  # Exclude first two rows
                        cell.text = dfTot[slide_by].unique()[i - 2]
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(6)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                    if i == 1 and j > 0:
                        text_frame = cell.text_frame

                        if cell.text == "Value Share (%)":
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                            cell.text_frame.paragraphs[0].font.size = Pt(7)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                        if cell.text != "Value Share (%)":
                            cell.text = cell.text.replace('Neumarkt', clientBrand).replace('| WOB', '')
                            paragraph = text_frame.paragraphs[0]
                            run_2 = paragraph.add_run()
                            run_2.text = "| WOB"
                            run_2.font.color.rgb = RGBColor(0, 160, 151)
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                            cell.text_frame.paragraphs[0].font.size = Pt(7)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            for chartNum in range(12):
                if chartNum > (2 * tablelength + 1):
                    delete_table_chart(slide, charts[chartNum])
                if chartNum == 0:
                    chart = charts[chartNum].chart
                    chart_data = CategoryChartData()
                    chart_data.categories = ['']
                    dfTot['Value Share recal'] = dfTot['Value Share recal'].astype(float)
                    dfTot['Value Share recal'] = np.where(round(dfTot['Value Share recal'].replace(np.nan, 0), 3) < .0005, None, dfTot['Value Share'])                   
                    series = dfTot['Value Share recal'].replace(np.nan, None)
                    chart_data.add_series(category, series)
                    chart.replace_data(chart_data)
                    value_axis = chart.value_axis
                    value_axis.minimum_scale = 0
                    value_axis.maximum_scale = maxaxis
                    xlsx_file = BytesIO()
                    with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                        chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                        worksheet.write(0, 0, f"{slide_by}")
                        worksheet.write_column(1, 0, dfTot[slide_by], None)
                    chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())
                if chartNum == 1:
                    chart = charts[chartNum].chart
                    chart_data = CategoryChartData()
                    chart_data.categories = ['']
                    dfTotParent['Value Share'] = dfTotParent['Value Share'].astype(float)
                    dfTotParent['Value Share'] = np.where(round(dfTotParent['Value Share'].replace(np.nan, 0), 3) < .0005, None, dfTotParent['Value Share'])
                    series = dfTotParent['Value Share'].replace(np.nan, None)
                    chart_data.add_series(namepar, series)
                    chart.replace_data(chart_data)
                    value_axis = chart.value_axis
                    value_axis.minimum_scale = 0
                    value_axis.maximum_scale = maxaxis
                    xlsx_file = BytesIO()
                    with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                        chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                        worksheet.write(0, 0, f"{slide_by}")
                        worksheet.write_column(1, 0, dfTotParent[slide_by], None)
                    chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

            for secnum, sec in enumerate(scope):
                if math.floor(secnum / 5) == slidenum:
                    secnum = secnum - 5 * slidenum
                    dfSectorBrand = pd.DataFrame()
                    dfSectors = pd.DataFrame()
                    dfSectorBrand = ToplineBracketSector[market][(ToplineBracketSector[market][slideType] == sec) & (ToplineBracketSector[market][parent] == namepar) & (ToplineBracketSector[market]["Top Brands"] == clientBrand)]
                    dfSectorBrand['Size'] = dfSectorBrand[slide_by].apply(extract_numeric_value)
                    # dfSectorBrand = dfSectorBrand[dfSectorBrand['Size'] != 0]
                    dfSectorBrand['Scope'] = sec + " " + clientBrand
                    dfSectors = dfSectorsTot[dfSectorsTot[slideType] == sec]

                    # Missing brackets scope vs category
                    missingTotalBracket = list(set(dfTot[slide_by].unique()) - set(dfSectors[slide_by].unique()))
                    missingTotalBracket = pd.DataFrame({slide_by: missingTotalBracket, 'Size': [extract_numeric_value(x) for x in missingTotalBracket]})
                    dfSectors = pd.concat([dfSectors, missingTotalBracket]).sort_values(by='Size', ascending=False)
                    # Missing brackets brands vs category
                    missingBrandBracket = list(set(dfTot[slide_by].unique()) - set(dfSectorBrand[slide_by].unique()))
                    missingBrandBracket = pd.DataFrame({slide_by: missingBrandBracket, 'Size': [extract_numeric_value(x) for x in missingBrandBracket]})
                    dfSectorBrand = pd.concat([dfSectorBrand, missingBrandBracket]).sort_values(by='Size', ascending=False)

                    
                for i, row in enumerate(table0.rows):
                    for j, cell in enumerate(row.cells):
                        if i>1 and j==(secnum)*2+4 and tablelength>=secnum+1:
                            if manufOrbrand=="Top Brands":
                                value='' if (dfSectorBrand['Brand WoB %'].reset_index(drop=True).astype(str)[i-2]=='nan' or dfSectorBrand['Value Share'].reset_index(drop=True)[i-2]<0.0005) else str(int(round(dfSectorBrand['Brand WoB %'].reset_index(drop=True).astype(float)[i-2]*100,0)))+'%'
                            else:
                                value='' if (dfSectorBrand['Company WoB %'].reset_index(drop=True).astype(str)[i-2]=='nan' or dfSectorBrand['Value Share'].reset_index(drop=True)[i-2]<0.0005) else str(int(round(dfSectorBrand['Company WoB %'].reset_index(drop=True).astype(float)[i-2]*100,0)))+'%'

                            cell.text='' if (value =='0.0%' or value =='0%') else value
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Book Italic'
                            cell.text_frame.paragraphs[0].font.size = Pt(6)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.RIGHT
                            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0,160,151)
                            cell.text_frame.paragraphs[0].font.bold = True

                for chartNum in range(11):
                    if chartNum > 1 and chartNum - 2 * secnum == 2:
                        chart = charts[chartNum].chart
                        chart_data = CategoryChartData()
                        chart_data.categories = ['']
                        dfSectors['Value Share'] = dfSectors['Value Share'].astype(float)
                        dfSectors['Value Share'] = np.where(round(dfSectors['Value Share'].replace(np.nan, 0), 3) < .0005, None, dfSectors['Value Share'])
                        series = dfSectors['Value Share'].replace(np.nan, None)
                        chart_data.add_series(sec, series)
                        chart.replace_data(chart_data)
                        value_axis = chart.value_axis
                        value_axis.minimum_scale = 0
                        value_axis.maximum_scale = maxaxis

                        xlsx_file = BytesIO()
                        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                            worksheet.write(0, 0, f"{slide_by}")
                            worksheet.write_column(1, 0, dfSectors[slide_by], None)
                        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

                    if chartNum > 1 and chartNum - 2 * secnum == 3:
                        chart = charts[chartNum].chart
                        chart_data = CategoryChartData()
                        chart_data.categories = ['']
                        dfSectorBrand['Value Share'] = dfSectorBrand['Value Share'].astype(float)
                        dfSectorBrand['Value Share'] = np.where(round(dfSectorBrand['Value Share'].replace(np.nan, 0), 3) < .0005, None, dfSectorBrand['Value Share'])
                        series = dfSectorBrand['Value Share'].replace(np.nan, None)
                        chart_data.add_series(sec, series)
                        chart.replace_data(chart_data)
                        value_axis = chart.value_axis
                        value_axis.minimum_scale = 0
                        value_axis.maximum_scale = maxaxis
                        xlsx_file = BytesIO()
                        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                            worksheet.write(0, 0, f"{slide_by}")
                            worksheet.write_column(1, 0, dfSectorBrand[slide_by], None)
                        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

<>:73: SyntaxWarning: invalid escape sequence '\d'
<>:74: SyntaxWarning: invalid escape sequence '\d'
<>:75: SyntaxWarning: invalid escape sequence '\d'
<>:85: SyntaxWarning: invalid escape sequence '\d'
<>:231: SyntaxWarning: invalid escape sequence '\d'
<>:238: SyntaxWarning: invalid escape sequence '\d'
<>:242: SyntaxWarning: invalid escape sequence '\d'
<>:73: SyntaxWarning: invalid escape sequence '\d'
<>:74: SyntaxWarning: invalid escape sequence '\d'
<>:75: SyntaxWarning: invalid escape sequence '\d'
<>:85: SyntaxWarning: invalid escape sequence '\d'
<>:231: SyntaxWarning: invalid escape sequence '\d'
<>:238: SyntaxWarning: invalid escape sequence '\d'
<>:242: SyntaxWarning: invalid escape sequence '\d'
C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_23284\2341702956.py:73: SyntaxWarning: invalid escape sequence '\d'
  dfTot['Size']=dfTot[slide_by].apply(lambda x:x.split('-')[1].split(' ')[0] if '-' in x else 9999).astype(str).str.extract('(\d+)').astype(float)
C:\Users\S

#### Slide ISD

In [ ]:
def interSizeDiscount(prs, dfVariant, key, percent, ValueCutOff, numOfDuplicates, ISDcolumn, position=0):
    """
    Function to adjust and update PowerPoint slides with data from a DataFrame.

    Parameters:
    prs (Presentation): PowerPoint presentation object.
    dfVariant (DataFrame): DataFrame containing variant data.
    key (str): Key information.
    percent (float): Percentage value.
    ValueCutOff (float): Value cutoff.
    numOfDuplicates (int): Number of duplicates.
    position (int, optional): Position parameter. Default is 0.

    Returns:
    None
    """
    # Iterate over the specified number of duplicates
    for slidenum in range(numOfDuplicates):
        # Get the unique variant for the current slide number
        variant = dfVariant['Variant'].unique()[slidenum]
        
        # Filter DataFrame by the current variant
        df = dfVariant[dfVariant['Variant'] == variant]
        
        # Clean 'Total Size' column and convert to integer
       
        df['Size'] = df['Total Size'].str.extract('(\d+)', expand=False).astype(float)
        
        # Sort DataFrame by 'Size'
        df = df.sort_values(by=['Size'], ascending=True)

        # Filter DataFrame by 'Value Sales' cutoff and reset index
        df = df[df['Value Sales'] >= ValueCutOff].reset_index(drop=True)
        
        # Find anchor values for calculations
        anchorValue = df[df['Unit Sales'] == df['Unit Sales'].max()]
        anchorBasePrice = df[df['Unit Sales'] == df['Unit Sales'].max()]['Base Price/Unit'].iloc[0].replace(currency, '').strip()

        anchorSize = df[df['Unit Sales'] == df['Unit Sales'].max()]['Total Size No']
        
        # Calculate linear price based on anchor values
        df['Linear Price Calculated'] = (float(anchorBasePrice) / anchorSize.iloc[0]) * df['Total Size No']

        # Update text in shapes
        shapes = prs.slides[slidenum + position].shapes

        axisNumber = get_shape_number(shapes, 'Value \nSales \n(M)')
        currencyNumber = get_shape_number(shapes, 'Shelf \nPrice\n/Unit\n(€)')
        titleNumber = get_shape_number(shapes, "Inter-size Discount  | Variant | National | P12M")
        sourceNumber = get_shape_number(shapes, "DATA SOURCE: Trade Panel/Retailer Data | Ending May 2023")

        shapes[sourceNumber].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('National', key.split(' | ')[2]).replace('Variant', variant).replace('P12M', period)
        shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[axisNumber].text = shapes[axisNumber].text.replace('M', percentstr)
        shapes[currencyNumber].text = shapes[currencyNumber].text.replace('€', currency)
        
        # Update formatting for text
        for paragraph in shapes[axisNumber].text_frame.paragraphs:
            paragraph.font.size = Pt(8)
            paragraph.font.name = 'Nexa Bold'
            paragraph.font.color.rgb = RGBColor(87, 85, 85)
            paragraph.alignment = PP_ALIGN.RIGHT
        for paragraph in shapes[currencyNumber].text_frame.paragraphs:
            paragraph.font.size = Pt(8)
            paragraph.font.name = 'Nexa Bold'
            paragraph.font.color.rgb = RGBColor(87, 85, 85)
            paragraph.alignment = PP_ALIGN.LEFT
        
        # Create table and chart
        tables, charts = createTableAndChart(shapes)
        table = tables[0].table

        ## Adjust Table Row numbers
        num_columns_to_remove = (len(table.columns) - df.shape[0]) - 1  # Specify the number of rows to remove from the end
#         table_width = Inches(8.8)  # Specify the desired table height
        table = col_cell_remove(table, num_columns_to_remove)
                
        # Replace the table data
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if j == 0:
                    continue
                if i == 0:
                    val = list(df['Value Sales IYA'])[j - 1]
                    cell.text = '' if str(val) == '0' else str(val)
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 0, 0) if val < 100 else RGBColor(87, 85, 85)
                if i == 1:
                    cell.text = str(list(df['VSOD'])[j - 1])
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        
        # Update chart data
        chart = charts[0].chart
#         charts[0].left = Inches(.58)
        chart_data = CategoryChartData()
        chart_data.categories = df[ISDcolumn]
        chart_data.add_series('Value Sales', (df['Value Sales'] / percent))
        chart_data.add_series('Shelf Price/Unit', df['Base Price/Unit'].str.replace(f'{currency}', '').astype(float))
        chart_data.add_series('Linear Price', df['Linear Price Calculated'])
        
        basePrice = list(df['Base Price/Unit'].str.replace(f'{currency}', '').astype(float))
        linearPrice = list(df['Linear Price Calculated'])
        interDiscountPos = [(basePrice[i] + linearPrice[i]) / 2 for i in range(len(linearPrice))]
        chart_data.add_series('Inter Size Discount', interDiscountPos)
        
        df['Discount Calculated'] = df['Base Price/Unit'].str.replace(f'{currency}', '').astype(float) / df['Linear Price Calculated']
        interDiscountVal = list(df['Discount Calculated'])
        chart.replace_data(chart_data)
        value_axis = chart.value_axis
        value_axis.minimum_scale = 0
        value_axis.maximum_scale = int((df['Value Sales'] / percent).max()) * 2
        
        # Update data labels
        for idx, point in enumerate(chart.series[3].points):
            data_label = point.data_label
            data_label.has_text_frame = True
            data_label.text_frame.text = str(int(round((interDiscountVal[idx] - 1) * 100, 0))) + '%'
            data_label.position = XL_LABEL_POSITION.CENTER
            
            data_label_0 = chart.series[0].points[idx].data_label
            data_label_0.has_text_frame = True
            data_label_0.text_frame.text = ''
            if idx == anchorValue.index[0]:
                data_label_0.has_text_frame = True
                data_label_0.text_frame.text = 'Anchor'
                data_label_0.position = XL_LABEL_POSITION.OUTSIDE_END
                font = data_label_0.text_frame.paragraphs[0].runs[0].font
                font.name = "Nexa Book (Body)"
                font.size = Pt(11)
                font.color.rgb = RGBColor(0, 160, 151)
                
            data_label_1 = chart.series[1].points[idx].data_label
            data_label_1.has_text_frame = True
            data_label_1.text_frame.text = str(round(df['Base Price/Unit'].str.replace(f'{currency}', '').astype(float).iloc[idx], 2))
            data_label_1.position = XL_LABEL_POSITION.ABOVE
        
        # Update chart data and format
        chart.replace_data(chart_data)
        secondaryValueAxes(chart,formatVal='#,##0.00')
